# Mask generator using Dino+SAM

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
# Reading the entire file for coco text
text = ""
with open('coco-labels-2014_2017.txt', 'r') as file:
#     content = file.read()
    for line in file:
#         print(line, len(line)) 
        text += "."+line.strip()
text = text[1:]

In [5]:
# cityscape list
text = [
    {"name": "unlabeled", "color": "#000000", "attributes": []},
    {"name": "egovehicle", "color": "#000000", "attributes": []},
    {"name": "rectificationborder", "color": "#000000", "attributes": []},
    {"name": "outofroi", "color": "#000000", "attributes": []},
    {"name": "static", "color": "#000000", "attributes": []},
    {"name": "dynamic", "color": "#6f4a00", "attributes": []},
    {"name": "ground", "color": "#510051", "attributes": []},
    {"name": "road", "color": "#804080", "attributes": []},
    {"name": "sidewalk", "color": "#f423e8", "attributes": []},
    {"name": "parking", "color": "#faaaa0", "attributes": []},
    {"name": "railtrack", "color": "#e6968c", "attributes": []},
    {"name": "building", "color": "#464646", "attributes": []},
    {"name": "wall", "color": "#66669c", "attributes": []},
    {"name": "fence", "color": "#be9999", "attributes": []},
    {"name": "guardrail", "color": "#b4a5b4", "attributes": []},
    {"name": "bridge", "color": "#966464", "attributes": []},
    {"name": "tunnel", "color": "#96785a", "attributes": []},
    {"name": "pole", "color": "#999999", "attributes": []},
    {"name": "polegroup", "color": "#999999", "attributes": []},
    {"name": "trafficlight", "color": "#faaa1e", "attributes": []},
    {"name": "trafficsign", "color": "#dcdc00", "attributes": []},
    {"name": "vegetation", "color": "#6b8e23", "attributes": []},
    {"name": "terrain", "color": "#98fb98", "attributes": []},
    {"name": "sky", "color": "#4682b4", "attributes": []},
    {"name": "person", "color": "#dc143c", "attributes": []},
    {"name": "rider", "color": "#ff0000", "attributes": []},
    {"name": "car", "color": "#00008e", "attributes": []},
    {"name": "truck", "color": "#000046", "attributes": []},
    {"name": "bus", "color": "#003c64", "attributes": []},
    {"name": "caravan", "color": "#00005a", "attributes": []},
    {"name": "trailer", "color": "#00006e", "attributes": []},
    {"name": "train", "color": "#005064", "attributes": []},
    {"name": "motorcycle", "color": "#0000e6", "attributes": []},
    {"name": "bicycle", "color": "#770b20", "attributes": []},
    {"name": "licenseplate", "color": "#00000e", "attributes": []}
]

annot_list = text[0]["name"]
for t in text[1:]:
    annot_list += "."+ t["name"]
annot_list

'unlabeled.egovehicle.rectificationborder.outofroi.static.dynamic.ground.road.sidewalk.parking.railtrack.building.wall.fence.guardrail.bridge.tunnel.pole.polegroup.trafficlight.trafficsign.vegetation.terrain.sky.person.rider.car.truck.bus.caravan.trailer.train.motorcycle.bicycle.licenseplate'

In [1]:
import sys
sys.path.append('/mnt/nas/jiaojiao/2024/gen_ss')

import argparse
import os
import copy

import numpy as np
import json
import torch
from PIL import Image, ImageDraw, ImageFont

# Grounding DINO
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap, get_phrases_from_posmap_2

# segment anything
from segment_anything import build_sam, SamPredictor 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
from tqdm import tqdm

# class palette for PASCAL
palette = [0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 128, 0, 0, 0, 128, 128, 0, 128, 0, 128, 128, 128, 128, 128,
           64, 0, 0, 192, 0, 0, 64, 128, 0, 192, 128, 0, 64, 0, 128, 192, 0, 128, 64, 128, 128, 192, 128, 128,
           0, 64, 0, 128, 64, 0, 0, 192, 0, 128, 192, 0, 0, 64, 128, 128, 64, 128, 0, 192, 128, 128, 192, 128,
           64, 64, 0, 192, 64, 0, 64, 192, 0, 192, 192, 0]

classes = ['aeroplane','bicycle','bird', 'boat','bottle','bus','car','cat','chair',
           'cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep',
           'sofa','train','tvmonitor']

class_dict = dict(zip(classes, list(range(1, 21))))

# print(class_dict)

class_tree = {}
# add some propmt here:
# class_tree['person'] = ['']
class_tree['horse'] = ['halter', 'saddle']
class_tree['diningtable'] = ['bowl','plate','food','fruit','glass', 'dishes']
class_tree['tvmonitor'] = ['tv', 'monitor']
class_tree['sofa'] = ['couch']
class_tree['bottle'] = ['wine bottle', 'water bottle', 'canister']
class_tree['pottedplant'] = ['couch']

for class_name, class_idx in list(class_dict.items()):
    if class_name in class_tree:
        sub_class_list = class_tree[class_name]
        for sub_class in sub_class_list:
            class_dict[sub_class] = class_idx

print(class_dict)
print(class_tree)


# Todo: add coco palette
palette = [0,0,0,
        220, 20, 60, 119, 11, 32, 0, 0, 142, 0, 0, 230, 106, 0, 228, #5
        0, 60, 100, 0, 80, 100, 0, 0, 70, 0, 0, 192, 250, 170, 30,
        100, 170, 30, 220, 220, 0, 175, 116, 175, 250, 0, 30, 165, 42, 42,
        255, 77, 255, 0, 226, 252, 182, 182, 255, 0, 82, 0, 120, 166, 157,
        110, 76, 0, 174, 57, 255, 199, 100, 0, 72, 0, 118, 255, 179, 240,
        0, 125, 92, 209, 0, 151, 188, 208, 182, 0, 220, 176, 255, 99, 164,
        92, 0, 73, 133, 129, 255, 78, 180, 255, 0, 228, 0, 174, 255, 243,
        45, 89, 255, 134, 134, 103, 145, 148, 174, 255, 208, 186, 197, 226, 255,
        171, 134, 1, 109, 63, 54, 207, 138, 255, 151, 0, 95, 9, 80, 61,
        84, 105, 51, 74, 65, 105, 166, 196, 102, 208, 195, 210, 255, 109, 65, #50
        0, 143, 149, 179, 0, 194, 209, 99, 106, 5, 121, 0, 227, 255, 205,
        147, 186, 208, 153, 69, 1, 3, 95, 161, 163, 255, 0, 119, 0, 170,
        0, 182, 199, 0, 165, 120, 183, 130, 88, 95, 32, 0, 130, 114, 135,
        110, 129, 133, 166, 74, 118, 219, 142, 185, 79, 210, 114, 178, 90, 62,
        65, 70, 15, 127, 167, 115, 59, 105, 106, 142, 108, 45, 196, 172, 0,
        95, 54, 80, 128, 76, 255, 201, 57, 1, 246, 0, 122, 191, 162, 208
    ]

classes = [{"supercategory": "person", "id": 1, "name": "person"}, # 一共80类
               {"supercategory": "vehicle", "id": 2, "name": "bicycle"},
               {"supercategory": "vehicle", "id": 3, "name": "car"},
               {"supercategory": "vehicle", "id": 4, "name": "motorcycle"},
               {"supercategory": "vehicle", "id": 5, "name": "airplane"},
               {"supercategory": "vehicle", "id": 6, "name": "bus"},
               {"supercategory": "vehicle", "id": 7, "name": "train"},
               {"supercategory": "vehicle", "id": 8, "name": "truck"},
               {"supercategory": "vehicle", "id": 9, "name": "boat"},
               {"supercategory": "outdoor", "id": 10, "name": "traffic light"},
               {"supercategory": "outdoor", "id": 11, "name": "fire hydrant"},
               {"supercategory": "outdoor", "id": 13, "name": "stop sign"},
               {"supercategory": "outdoor", "id": 14, "name": "parking meter"},
               {"supercategory": "outdoor", "id": 15, "name": "bench"},
               {"supercategory": "animal", "id": 16, "name": "bird"},
               {"supercategory": "animal", "id": 17, "name": "cat"},
               {"supercategory": "animal", "id": 18, "name": "dog"},
               {"supercategory": "animal", "id": 19, "name": "horse"},
               {"supercategory": "animal", "id": 20, "name": "sheep"},
               {"supercategory": "animal", "id": 21, "name": "cow"},
               {"supercategory": "animal", "id": 22, "name": "elephant"},
               {"supercategory": "animal", "id": 23, "name": "bear"},
               {"supercategory": "animal", "id": 24, "name": "zebra"},
               {"supercategory": "animal", "id": 25, "name": "giraffe"},
               {"supercategory": "accessory", "id": 27, "name": "backpack"},
               {"supercategory": "accessory", "id": 28, "name": "umbrella"},
               {"supercategory": "accessory", "id": 31, "name": "handbag"},
               {"supercategory": "accessory", "id": 32, "name": "tie"},
               {"supercategory": "accessory", "id": 33, "name": "suitcase"},
               {"supercategory": "sports", "id": 34, "name": "frisbee"},
               {"supercategory": "sports", "id": 35, "name": "skis"},
               {"supercategory": "sports", "id": 36, "name": "snowboard"},
               {"supercategory": "sports", "id": 37, "name": "sports ball"},
               {"supercategory": "sports", "id": 38, "name": "kite"},
               {"supercategory": "sports", "id": 39, "name": "baseball bat"},
               {"supercategory": "sports", "id": 40, "name": "baseball glove"},
               {"supercategory": "sports", "id": 41, "name": "skateboard"},
               {"supercategory": "sports", "id": 42, "name": "surfboard"},
               {"supercategory": "sports", "id": 43, "name": "tennis racket"},
               {"supercategory": "kitchen", "id": 44, "name": "bottle"},
               {"supercategory": "kitchen", "id": 46, "name": "wine glass"},
               {"supercategory": "kitchen", "id": 47, "name": "cup"},
               {"supercategory": "kitchen", "id": 48, "name": "fork"},
               {"supercategory": "kitchen", "id": 49, "name": "knife"},
               {"supercategory": "kitchen", "id": 50, "name": "spoon"},
               {"supercategory": "kitchen", "id": 51, "name": "bowl"},
               {"supercategory": "food", "id": 52, "name": "banana"},
               {"supercategory": "food", "id": 53, "name": "apple"},
               {"supercategory": "food", "id": 54, "name": "sandwich"},
               {"supercategory": "food", "id": 55, "name": "orange"},
               {"supercategory": "food", "id": 56, "name": "broccoli"},
               {"supercategory": "food", "id": 57, "name": "carrot"},
               {"supercategory": "food", "id": 58, "name": "hot dog"},
               {"supercategory": "food", "id": 59, "name": "pizza"},
               {"supercategory": "food", "id": 60, "name": "donut"},
               {"supercategory": "food", "id": 61, "name": "cake"},
               {"supercategory": "furniture", "id": 62, "name": "chair"},
               {"supercategory": "furniture", "id": 63, "name": "couch"},
               {"supercategory": "furniture", "id": 64, "name": "potted plant"},
               {"supercategory": "furniture", "id": 65, "name": "bed"},
               {"supercategory": "furniture", "id": 67, "name": "dining table"},
               {"supercategory": "furniture", "id": 70, "name": "toilet"},
               {"supercategory": "electronic", "id": 72, "name": "tv"},
               {"supercategory": "electronic", "id": 73, "name": "laptop"},
               {"supercategory": "electronic", "id": 74, "name": "mouse"},
               {"supercategory": "electronic", "id": 75, "name": "remote"},
               {"supercategory": "electronic", "id": 76, "name": "keyboard"},
               {"supercategory": "electronic", "id": 77, "name": "cell phone"},
               {"supercategory": "appliance", "id": 78, "name": "microwave"},
               {"supercategory": "appliance", "id": 79, "name": "oven"},
               {"supercategory": "appliance", "id": 80, "name": "toaster"},
               {"supercategory": "appliance", "id": 81, "name": "sink"},
               {"supercategory": "appliance", "id": 82, "name": "refrigerator"},
               {"supercategory": "indoor", "id": 84, "name": "book"},
               {"supercategory": "indoor", "id": 85, "name": "clock"},
               {"supercategory": "indoor", "id": 86, "name": "vase"},
               {"supercategory": "indoor", "id": 87, "name": "scissors"},
               {"supercategory": "indoor", "id": 88, "name": "teddy bear"},
               {"supercategory": "indoor", "id": 89, "name": "hair drier"},
               {"supercategory": "indoor", "id": 90, "name": "toothbrush"}]

id_to_name_dict = {}
id_to_cls_dict = {}
name_to_cls_dict = {}
for index, item in enumerate(classes):
    category_id = item['id']
    category_name = item['name']
    id_to_name_dict[category_id] = category_name
    id_to_cls_dict[category_id] = index + 1
    name_to_cls_dict[category_name] = index + 1 
    
print("COCO name to class list: ", name_to_cls_dict)

# Define the COCO color palette (this is a common palette used for visualizing COCO dataset)
COCO_palette = [ 
    (220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228),
    (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30),
    (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30), (165, 42, 42),
    (255, 77, 255), (0, 226, 252), (182, 182, 255), (0, 82, 0), (120, 166, 157),
    (110, 76, 0), (174, 57, 255), (199, 100, 0), (72, 0, 118), (255, 179, 240),
    (0, 125, 92), (209, 0, 151), (188, 208, 182), (0, 220, 176), (255, 99, 164),
    (92, 0, 73), (133, 129, 255), (78, 180, 255), (0, 228, 0), (174, 255, 243),
    (45, 89, 255), (134, 134, 103), (145, 148, 174), (255, 208, 186), (197, 226, 255),
    (171, 134, 1), (109, 63, 54), (207, 138, 255), (151, 0, 95), (9, 80, 61),
    (84, 105, 51), (74, 65, 105), (166, 196, 102), (208, 195, 210), (255, 109, 65),
    (0, 143, 149), (179, 0, 194), (209, 99, 106), (5, 121, 0), (227, 255, 205),
    (147, 186, 208), (153, 69, 1), (3, 95, 161), (163, 255, 0), (119, 0, 170),
    (0, 182, 199), (0, 165, 120), (183, 130, 88), (95, 32, 0), (130, 114, 135),
    (110, 129, 133), (166, 74, 118), (219, 142, 185), (79, 210, 114), (178, 90, 62),
    (65, 70, 15), (127, 167, 115), (59, 105, 106), (142, 108, 45), (196, 172, 0),
    (95, 54, 80), (128, 76, 255), (201, 57, 1), (246, 0, 122), (191, 162, 208)
]
# Convert the color palette to a NumPy array for easier manipulation
COCO_palette_np = np.array(COCO_palette, dtype=np.uint8)

def apply_palette(image, palette):
    # Create a new image with the same dimensions as the input image
    color_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

    # Apply the color palette
    for label in np.unique(image):
        if label == 0:  # Assuming 0 is the background
            continue
        color_image[image == label] = [label]

    return Image.fromarray(color_image)

def load_image(image_path):
    # load image
    image_pil = Image.open(image_path).convert("RGB")  # load image

    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    image, _ = transform(image_pil, None)  # 3, h, w
    return image_pil, image

def load_model(model_config_path, model_checkpoint_path, device):
    args = SLConfig.fromfile(model_config_path)
    args.device = device
    model = build_model(args)
#     checkpoint = torch.load(model_checkpoint_path, map_location="cpu")
    checkpoint = torch.load(model_checkpoint_path, map_location=device) # debug
    load_res = model.load_state_dict(clean_state_dict(checkpoint["model"]), strict=False)
    # print(load_res)
    _ = model.eval()
    return model

def get_grounding_output(model, image, caption, box_threshold, text_threshold, with_logits=True, device="cpu"):
    caption = caption.lower()
    caption = caption.strip()
    if not caption.endswith("."):
        caption = caption + "."
    model = model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image[None], captions=[caption])
    # todo: debug
#     if device == "cpu":
    logits = outputs["pred_logits"].cpu().sigmoid()[0]  # (nq, 256)
    boxes = outputs["pred_boxes"].cpu()[0]  # (nq, 4)
#     else:
#         logits = outputs["pred_logits"].sigmoid()[0]  # (nq, 256)
#         boxes = outputs["pred_boxes"][0]  # (nq, 4)
    logits.shape[0]

    # filter output
    logits_filt = logits.clone()
    boxes_filt = boxes.clone()
    filt_mask = logits_filt.max(dim=1)[0] > box_threshold
    logits_filt = logits_filt[filt_mask]  # num_filt, 256
    boxes_filt = boxes_filt[filt_mask]  # num_filt, 4
    logits_filt.shape[0]

    # get phrase
    tokenlizer = model.tokenizer
    tokenized = tokenlizer(caption)
    
    # breakpoint()
    # build pred
    pred_phrases = []

    for logit, box in zip(logits_filt, boxes_filt):

        pred_phrase = get_phrases_from_posmap_2(logit > text_threshold, logit, tokenized, tokenlizer)

        if with_logits:
            pred_phrases.append(pred_phrase + f"({str(logit.max().item())[:4]})")
        else:
            pred_phrases.append(pred_phrase)

    return boxes_filt, pred_phrases

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_box(box, ax, label):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)) 
    ax.text(x0, y0, label)


def save_mask_data(output_dir, mask_list, box_list, label_list, name, H, W):
    value = 0  # 0 for background

    mask_img = torch.zeros(H, W)
    for idx, mask in enumerate(mask_list):
        class_name, logit = label_list[idx].split('(')
        if class_name in class_dict:
            class_idx = class_dict[class_name]
            mask_img[mask.cpu().numpy()[0] == True] = class_idx 
    

    out = mask_img.numpy().astype(np.uint8)
    out = Image.fromarray(out, mode='P')
    # for Pascal
    #     out.putpalette(palette)
    out_name = os.path.join(output_dir, '{}.png'.format(name))
    out.save(out_name)


    json_data = [{
        'value': value,
        'label': 'background'
    }]

    for label, box in zip(label_list, box_list):
        value += 1
        name, logit = label.split('(')
        logit = logit[:-1] # the last is ')'
        json_data.append({
            'value': value,
            'label': name,
            'logit': float(logit),
            'box': box.numpy().tolist(),
        })
    with open(os.path.join(output_dir, 'mask.json'), 'w') as f:
        json.dump(json_data, f)
    
    
def save_mask_data_coco(output_dir, mask_list, box_list, label_list, name, H, W):
    value = 0  # 0 for background

    mask_img = torch.zeros(H, W)
    for idx, mask in enumerate(mask_list):
        class_name, logit = label_list[idx].split('(')
       
        if class_name in name_to_cls_dict:
            class_idx = name_to_cls_dict[class_name]
            mask_img[mask.cpu().numpy()[0] == True] = class_idx 
    
    out = mask_img.numpy().astype(np.uint8)

    out_name = os.path.join(output_dir, '{}.png'.format(name))
    
    pseudo = Image.fromarray(out, mode='P')
    pseudo.save(out_name)
    
    out_name = out_name.replace('.png', '_color.png')
    # add palette
    pseudo.putpalette(palette) 
#     pseudo = apply_palette(np.array(pseudo), COCO_palette_np)
    
    # save one_hot vector
    pseudo.save(out_name)
#     #     out.putpalette(palette) # wrong colormap
    # save colored version
#     out_color = apply_palette(np.array(pseudo), COCO_palette_np)
#     out_name = os.path.join(output_dir, '{}.png'.format(name+"_color"))
#     out_color.save(out_name)

    json_data = [{
        'value': value,
        'label': 'background'
    }]

    for label, box in zip(label_list, box_list):
        value += 1
        name, logit = label.split('(')
        logit = logit[:-1] # the last is ')'
        json_data.append({
            'value': value,
            'label': name,
            'logit': float(logit),
            'box': box.numpy().tolist(),
        })
    with open(os.path.join(output_dir, 'mask.json'), 'w') as f:
        json.dump(json_data, f)
    

    return out

# if __name__ == "__main__":
def main(num):

#     parser = argparse.ArgumentParser("Grounded-Segment-Anything Demo", add_help=True)
#     parser.add_argument("--config", type=str, required=True, help="path to config file")
#     parser.add_argument(
#         "--grounded_checkpoint", type=str, required=True, help="path to checkpoint file"
#     )
#     parser.add_argument(
#         "--sam_checkpoint", type=str, required=True, help="path to checkpoint file"
#     )
#     parser.add_argument("--input_image", type=str, required=True, help="path to image file")
#     parser.add_argument("--img_list", type=str, required=True, default='metadata/pascal/train_aug(id).txt')
#     parser.add_argument("--im_path", default="/home/notebook/data/personal/S9050086/VOCdevkit/VOC2012/JPEGImages", type=str)


#     parser.add_argument(
#         "--output_dir", "-o", type=str, default="outputs", required=True, help="output directory"
#     )   

#     parser.add_argument("--box_threshold", type=float, default=0.3, help="box threshold")
#     parser.add_argument("--text_threshold", type=float, default=0.25, help="text threshold")

#     parser.add_argument("--device", type=str, default="cpu", help="running on cpu only!, default=False")
#     args = parser.parse_args()
    
    config = "../GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
    grounded_checkpoint = "../../wsss_sam/groundingdino_swint_ogc.pth" 
    sam_checkpoint = "/mnt/nas/jiaojiao/ckpt/sam/sam_vit_h_4b8939.pth" 
    box_threshold = 0.3 
    text_threshold = 0.25 
    device = "cuda:3" 
    datatype = "pascal" #["pascal", "cityscapes", “coco”]

    # data path
    if datatype == "pascal":
        img_list = "/mnt/nas/jiaojiao/2024/wsss_sam/metadata/pascal/train_aug(id).txt"  # source image
        im_path = "/mnt/nas/jiaojiao/data/VOCdevkit/VOC2012/JPEGImages"
        output_dir = "output/pascal/pascal_train" 
        with open(img_list) as f:
            img_list = []
            for line in f:
                img_list.append(line[:-1])
    elif datatype == "coco":
        # coco
        im_path = "/mnt/nas/jiaojiao/data/coco2014/images/train2014"
        output_dir = "output/coco/coco_train"
        img_list = os.listdir(im_path)
        img_list = "/mnt/nas/jiaojiao/2024/wsss_sam/metadata/coco/train.txt"
        with open(img_list) as f:
            img_list = []
            for line in f:
                img_list.append(line[:-1])
    elif datatype == "cityscapes":
        # cityscapes
        im_path = "/mnt/nas/kaichen/cityscape/leftImg8bit/train/stuttgart/"
        output_dir = "cityscapes_train_stuttgart" 
        img_list = os.listdir(im_path)
    
    cls_labels_dict = np.load('../metadata/cls_labels.npy',allow_pickle=True).item()
    
    print("size: ", len(img_list))
    # cfg
    config_file = config  # change the path of the model config file
    grounded_checkpoint = grounded_checkpoint  # change the path of the model
    sam_checkpoint = sam_checkpoint
    output_dir = output_dir
    box_threshold = box_threshold
    text_threshold = text_threshold
    device = device

    print("** Load Model **")
    # load model
    model = load_model(config_file, grounded_checkpoint, device=device)

    # make dir
    os.makedirs(output_dir, exist_ok=True)

    # initialize SAM
    predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

    print("** Load image **")
    j=0
    for index, name in tqdm(enumerate(img_list[:10])):

        if name+".png" in os.listdir(output_dir):
            continue
            
        # load image
        if datatype == "pascal" or "coco":
            image_path = os.path.join(im_path, '{}.jpg'.format(name))
        elif datatype == "cityscapes" :       
            image_path = os.path.join(im_path, name)
        
        image_pil, image = load_image(image_path)

        # get text prompt
        if datatype == "coco":
            text_prompt = text
            name = name.replace(".jpg", "")
        elif datatype == "cityscapes":
            text_prompt = "car.person.motorcycle" 
            text_prompt = annot_list # cityscapes
            name = name.replace(".png", "")
        elif datatype == "pascal":
            class_label = cls_labels_dict[name]
            text_prompt_list = []
            for i in range(20):
                if class_label[i] > 1e-5:
                    text_prompt_list.append(classes[i])
                    if classes[i] in list(class_tree.keys()):
                        sub_class_list = class_tree[classes[i]]
                        for sub_class in sub_class_list: 
                            text_prompt_list.append(sub_class)
        

    #         if 'pottedplant' not in text_prompt_list:
    #             continue

            if len(text_prompt_list) == 1:
                text_prompt = text_prompt_list[0]
            else:
                text_prompt = '.'.join(text_prompt_list)
            
        j+=1
        print(j,index, name,  '--', text_prompt)
#         print( text_prompt)

        # visualize raw image
        # image_pil.save(os.path.join(output_dir, "{}.jpg".format(name)))
        
        # run grounding dino model
        boxes_filt, pred_phrases = get_grounding_output(
            model, image, text_prompt, box_threshold, text_threshold, with_logits=True, device=device
        )

#         print("boxes_filt ", boxes_filt, "pred_phrases ", pred_phrases)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        predictor.set_image(image)

        size = image_pil.size
        H, W = size[1], size[0]
        for i in range(boxes_filt.size(0)):
            boxes_filt[i] = boxes_filt[i] * torch.Tensor([W, H, W, H])
            boxes_filt[i][:2] -= boxes_filt[i][2:] / 2
            boxes_filt[i][2:] += boxes_filt[i][:2]

        boxes_filt = boxes_filt.cpu()
        transformed_boxes = predictor.transform.apply_boxes_torch(boxes_filt, image.shape[:2]).to(device)

        try:
            masks, _, _ = predictor.predict_torch(
                point_coords = None,
                point_labels = None,
                boxes = transformed_boxes.to(device),
                multimask_output = False,
            )
        
        except: # in case nothing detected
            masks = []
        
        # print(pred_phrases)

        # draw output image
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
#         plt.show()

        for mask in masks:
            show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
        for box, label in zip(boxes_filt, pred_phrases):
            show_box(box.numpy(), plt.gca(), label)

        plt.axis('off')
        plt.savefig(
            os.path.join(output_dir, "{}_sam.jpg".format(name)), 
            bbox_inches="tight", dpi=300, pad_inches=0.0
        )

#         save_mask_data(output_dir, masks, boxes_filt, pred_phrases, name, H, W)
#         save_mask_data_coco(output_dir, masks, boxes_filt, pred_phrases, name, H, W)
        
        plt.show()
#         plt.close()



/home/jiaojiao/.conda/envs/fusion/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'aeroplane': 1, 'bicycle': 2, 'bird': 3, 'boat': 4, 'bottle': 5, 'bus': 6, 'car': 7, 'cat': 8, 'chair': 9, 'cow': 10, 'diningtable': 11, 'dog': 12, 'horse': 13, 'motorbike': 14, 'person': 15, 'pottedplant': 16, 'sheep': 17, 'sofa': 18, 'train': 19, 'tvmonitor': 20, 'wine bottle': 5, 'water bottle': 5, 'canister': 5, 'bowl': 11, 'plate': 11, 'food': 11, 'fruit': 11, 'glass': 11, 'dishes': 11, 'halter': 13, 'saddle': 13, 'couch': 18, 'tv': 20, 'monitor': 20}
{'horse': ['halter', 'saddle'], 'diningtable': ['bowl', 'plate', 'food', 'fruit', 'glass', 'dishes'], 'tvmonitor': ['tv', 'monitor'], 'sofa': ['couch'], 'bottle': ['wine bottle', 'water bottle', 'canister'], 'pottedplant': ['couch']}
COCO name to class list:  {'person': 1, 'bicycle': 2, 'car': 3, 'motorcycle': 4, 'airplane': 5, 'bus': 6, 'train': 7, 'truck': 8, 'boat': 9, 'traffic light': 10, 'fire hydrant': 11, 'stop sign': 12, 'parking meter': 13, 'bench': 14, 'bird': 15, 'cat': 16, 'dog': 17, 'horse': 18, 'sheep': 19, 'cow': 20, 

In [2]:
main(10) # coco: 82081

size:  10582
** Load Model **


/home/jiaojiao/.conda/envs/fusion/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 main(10) # coco: 82081                                                                       │
│   2                                                                                              │
│                                                                                                  │
│ in main:466                                                                                      │
│                                                                                                  │
│   463 │   os.makedirs(output_dir, exist_ok=True)                                                 │
│   464 │                                                                                          │
│   465 │   # initialize SAM                                                                       │
│ ❱ 466 │   predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))              │
│   467 │                                                                                          │
│   468 │   print("** Load image **")                                                              │
│   469 │   j=0                                                                                    │
│                                                                                                  │
│ /mnt/nas/jiaojiao/2024/wsss_sam/segment_anything/segment_anything/build_sam.py:20 in             │
│ build_sam_vit_h                                                                                  │
│                                                                                                  │
│    17 │   │   encoder_depth=32,                                                                  │
│    18 │   │   encoder_num_heads=16,                                                              │
│    19 │   │   encoder_global_attn_indexes=[7, 15, 23, 31],                                       │
│ ❱  20 │   │   checkpoint=checkpoint,                                                             │
│    21 │   )                                                                                      │
│    22                                                                                            │
│    23                                                                                            │
│                                                                                                  │
│ /mnt/nas/jiaojiao/2024/wsss_sam/segment_anything/segment_anything/build_sam.py:105 in _build_sam │
│                                                                                                  │
│   102 │   sam.eval()                                                                             │
│   103 │   if checkpoint is not None:                                                             │
│   104 │   │   with open(checkpoint, "rb") as f:                                                  │
│ ❱ 105 │   │   │   state_dict = torch.load(f)                                                     │
│   106 │   │   sam.load_state_dict(state_dict)                                                    │
│   107 │   return sam                                                                             │
│   108                                                                                            │
│                                                                                                  │
│ /home/jiaojiao/.conda/envs/fusion/lib/python3.7/site-packages/torch/serialization.py:789 in load │
│                                                                                                  │
│    786 │   │   │   │   │   │   return _load(opened_zipfile,